## Testing that polymerist is importable

In [1]:
import logging
logging.basicConfig(
    level=logging.INFO
)

import polymerist as ps
from polymerist.genutils.importutils import module_hierarchy

# print(module_hierarchy(ps))

INFO:numexpr.utils:Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:rdkit:Enabling RDKit 2023.09.5 jupyter extensions


[13:51:11] WARNING: not removing hydrogen atom with dummy atom neighbors
INFO:polymerist.smileslib.functgroups:Loading functional group SMARTS data from LUT


## Parameterizing PDB system

### Initializing paths and working directory

In [2]:
from pathlib import Path
from polymerist.genutils.fileutils.pathutils import assemble_path


working_dir = Path.cwd() # can change this to wherever your files are
mol_name = 'SPW_1m_JC'

pdb_path = assemble_path(working_dir, mol_name, extension='pdb') 
sdf_path = assemble_path(working_dir, mol_name, extension='sdf') 
inc_path = assemble_path(working_dir, mol_name, extension='pkl') 

### Load Topology from PDB ( no overhead since not using long molecules), check that molecules exist

In [3]:
from openff.toolkit import Molecule, Topology, ForceField

from polymerist.mdtools.openfftools import topology


if sdf_path.exists():
    offtop = topology.topology_from_sdf(sdf_path)
else:
    assert(pdb_path.exists())
    offtop = Topology.from_pdb(pdb_path)

mol = next(offtop.molecules)
mol.visualize(backend='nglview')

INFO:polymerist.mdtools.openfftools.topology:Loading serialized SDF Topology from /home/bamo6610/Documents/openff-dev/SPW_openMM/SPW_1m_JC.sdf


NGLWidget()

### Parameterize system w/ OpenFF, save relevant files to avoid doing this step more than once

In [4]:
import pickle
from openff.interchange import Interchange

from polymerist.unitutils.interop import openff_to_openmm
from polymerist.mdtools.openfftools import boxvectors
from polymerist.mdtools.openfftools import TKREGS, FF_PATH_REGISTRY


# load interchange from file if already extant, otherwise make a new one and save it
if inc_path.exists():
    with inc_path.open('rb') as file:
        inc  = pickle.load(file)
else:
    ff = ForceField('openff-2.0.0.offxml', 'tip3p.offxml') # load generic Sage + TIP3P force fields
    inc = ff.create_interchange(topology=offtop, toolkit_registry=TKREGS['OpenEye Toolkit']) # convert to interchange for export
    inc.box = boxvectors.get_topology_bbox(offtop)

    if not sdf_path.exists():
        topology.topology_to_sdf(sdf_path, inc.topology)

    with inc_path.open('wb') as file:
        pickle.dump(inc, file)

# extract OpenMM-specific objects to initialize simulations later
ommtop = inc.to_openmm_topology()
ommsys = inc.to_openmm_system(combine_nonbonded_forces=False, add_constrained_forces=True)
ommpos = openff_to_openmm(inc.positions)

In [17]:
from openmm.unit import kilojoule_per_mole, nanometer
import openmm
k=4184*(kilojoule_per_mole/nanometer**2)
droff=2.4*nanometer
fb_force = openmm.CustomExternalForce('k * (px^2); \
                               px = max(0, delta); \
                               delta = r - droff; \
                               r = abs(periodicdistance(x, y, z, x, y, z0));')
fb_force.addGlobalParameter('k', k)
fb_force.addGlobalParameter('droff', droff)
fb_force.addPerParticleParameter('z0')

0

## Creating OpenMM simulations

### Defining simulation parameters

In [5]:
# from polymerist.mdtools.openmmtools.parameters import SimulationParameters, ThermoParameters, IntegratorParameters, ReporterParameters
# from polymerist.mdtools.openmmtools.thermo import EnsembleFactory

# from openmm.unit import kelvin, atmosphere, nanosecond, picosecond, femtoseconds


# all_omm_sims : dict[str, SimulationParameters] = {
#     'equil_sim' : SimulationParameters(
#         integ_params=IntegratorParameters(
#             time_step=2*femtoseconds,
#             total_time=10*picosecond,
#             num_samples=50,
#         ),
#         thermo_params=ThermoParameters(
#             ensemble='NVT',
#             temperature=300 * kelvin,
#         ),
#         reporter_params=ReporterParameters(),
#     ),
#     'prod_sim' : SimulationParameters(
#         integ_params=IntegratorParameters(
#             time_step=2*femtoseconds,
#             total_time=100*picosecond,
#             num_samples=50,
#         ),
#         thermo_params=ThermoParameters(
#             ensemble='NVT',
#             temperature=300 * kelvin,
#         ),
#         reporter_params=ReporterParameters(),
#     ),
# }    

# # for path_name, sim_params in all_omm_sims.items():
#     # sim_params.to_file(assemble_path(working_dir, path_name, postfix='params', extension='json'))

### Run the defined simulation in series

In [6]:
# from polymerist.mdtools.openmmtools.execution import run_simulation_schedule


# omm_sim_dir = Path('OpenMM_sims')
# omm_sim_dir.mkdir(exist_ok=True)

# history = run_simulation_schedule(
#     working_dir=omm_sim_dir,
#     schedule=all_omm_sims,
#     init_top=ommtop,
#     init_sys=ommsys,
#     init_pos=ommpos,
#     return_history=True
)

INFO:polymerist.mdtools.openmmtools.execution:Initializing simulation 1/2 ("equil_sim")
INFO:polymerist.mdtools.openmmtools.thermo:Created LangevinMiddleIntegrator for NVT (Canonical) ensemble
INFO:polymerist.mdtools.openmmtools.reporters:Prepared DCDReporter which reports to OpenMM_sims/equil_sim/equil_sim_trajectory.dcd
INFO:polymerist.mdtools.openmmtools.reporters:Prepared CheckpointReporter which reports to OpenMM_sims/equil_sim/equil_sim_checkpoint.chk
INFO:polymerist.mdtools.openmmtools.reporters:Prepared StateReporter which reports to OpenMM_sims/equil_sim/equil_sim_state.xml
INFO:polymerist.mdtools.openmmtools.reporters:Prepared StateDataReporter which reports to OpenMM_sims/equil_sim/equil_sim_state_data.csv
INFO:polymerist.mdtools.openmmtools.preparation:Setting positions in Context
INFO:polymerist.mdtools.openmmtools.execution:Performing energy minimization (initial PE = 2.340798834608247e+17 kJ/mol)
INFO:polymerist.mdtools.openmmtools.execution:Energy successfully minimized

In [7]:
# sim_paths = history['prod_sim']['paths']

In [8]:
# sim_paths

SimulationPaths(parameters_path=PosixPath('OpenMM_sims/prod_sim/prod_sim_parameters.json'), paths_path=PosixPath('OpenMM_sims/prod_sim/prod_sim_paths.json'), system_path=PosixPath('OpenMM_sims/prod_sim/prod_sim_system.xml'), topology_path=PosixPath('OpenMM_sims/prod_sim/prod_sim_topology.pdb'), state_path=PosixPath('OpenMM_sims/prod_sim/prod_sim_state.xml'), checkpoint_path=PosixPath('OpenMM_sims/prod_sim/prod_sim_checkpoint.chk'), trajectory_path=PosixPath('OpenMM_sims/prod_sim/prod_sim_trajectory.dcd'), state_data_path=PosixPath('OpenMM_sims/prod_sim/prod_sim_state_data.csv'), time_data_path=None, spatial_data_path=None)